<a href="https://colab.research.google.com/github/felipemelonunes09/2024.Q2-PLN/blob/main/ai_chatbot_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicão LLM

Este notebook apresentará um guia abrangente para a construção de uma aplicação utilizando uma LLM. O processo envolve o uso de carregadores de documentos, embeddings, bancos de dados vetoriais e templates de prompt, utilizando principalmente o LangChain e a API da OpenAI. Esta abordagem enfatiza o uso da injeção de contexto em vez do fine-tuning, para conseguir respostas mais coerentes com o contexto fornecido.

## Injeção de Contexto

Ao invés de utilizar fine-tuning, essa abordagem é uma solução mais eficiente para a aplicação de conhecimento específico de domínio em LLMs. Os modelos pré-treinados estão restritos aos dados com os quais foram treinados, enquanto a injeção de contexto insere o contexto necessário dentro do próprio prompt, resultando em respostas mais precisas.

## 1. Carregando Documentos

Não utilizaremos nenhum carregador do LangChain. Em vez disso, faremos uso exclusivo da biblioteca BeautifulSoup para realizar o scraping de uma página web e extrair as informações necessárias para a nossa aplicação.



### Instalando Dependências



In [1]:
pip install beautifulsoup4

In [2]:
pip install requests

### Código

Realizaremos a requisição e transferiremos a resposta para um objeto da biblioteca BeautifulSoup.

In [3]:
import requests
from bs4 import BeautifulSoup

url       = "https://en.wikipedia.org/wiki/GPT-4"
response  = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

text = soup.get_text()



Selecionando a div de conteúdo e removendo as tags desnecessárias.

In [7]:
content_div = soup.find('div', {  'class': 'mw-parser-output' })

unwanted_tags = ['sup', 'span', 'table', 'ul', 'ol']
for tag in unwanted_tags:
    for match in content_div.findAll(tag):
        match.extract()

print(content_div.get_text()[0:100])


2023 text-generating language model



Generative Pre-trained Transformer 4 (GPT-4) is a multimodal 


## 2. Dividindo o Documento em Fragmentos de Texto

Agora, precisamos dividir o nosso texto em porções chamadas "text chunks". Isso nos permitirá comparar similaridades entre diferentes partes do texto.

### Instalando Dependências




In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


### Código

Esta seção cria esses fragmentos de texto, permitindo definir tanto o tamanho quanto a função de cálculo do tamanho com a classe RecursiveCharacterTextSplitter.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

article_text = content_div.get_text()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size    = 100,
    chunk_overlap = 20,
    length_function = len
)

texts = text_splitter.create_documents([article_text])
print(texts[0])
print(texts[1])
print(texts[2])

page_content='2023 text-generating language model'
page_content='Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by'
page_content='model created by OpenAI, and the fourth in its series of GPT foundation models. It was launched on'


## 3. De Text Chunks para Embeddings

Embora o texto seja legível para os humanos, é essencial convertê-lo para uma representação que possa ser interpretada e processada por máquinas, como bits e bytes.


Existem várias maneiras de obter essa representação. Neste contexto, buscamos um método para comparar esses fragmentos de texto e calcular o grau de proximidade entre eles.


Nesta parte iremos utlizar um Embedding Models provivionado por um serviço da OpenAI, com isso procuramos pegar nossos pedaços de texto e transformar em vetores, no modelo escolhido "Ada" nosso vetores terão 1536 dimencões, outros modelos podem ter valores diferentes para esse vetores.

### Aviso

Agora estamos lindando com um serviço terceiro pago, ou seja é necessário criar uma chave de API para a api da OpenAI e definir no painel o modelo que essa determinada chave pode utilizar.


### Instalando Depêndencias

In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


### Código

Nesta etapa atribuimos a chave da API para uma variável de ambiente

In [ ]:
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


E então utilizamos a funcionalidade de embeddings da API, e podemos verificar que ao fim o tamanho da embedding realmente corresponde valor do vetor de 1526 dimensões.

In [ ]:
import openai


embedding = openai.embeddings.create( input=texts[0].page_content, model="text-embedding-ada-002")
embedding = embedding.data[0].embedding

print(len(embedding))

1536


Agora com a habilidade de representar pedaços de texto e a pergunta do usuário como vetores é possivel verificar a similaridade entre dois pontos de dados, e para isso precisamos calcular a proximidade deles em um plano multidimensional.

Em nosso exemplo, será utilizado Cosine Similarity

Como queremos aplicar isso para todos os nossos text chunks, precisamos armazenar em uma estrutura mais robusta.

Começamos definindo um metodo para pegar um texto qualquer e transformar em um embedding.

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
  text = text.replace("\n", " ")
  return openai.embeddings.create( input=text, model=model).data[0].embedding

E então instalamos o pandas e o numpy para conseguirmos armazenar e fazer operações de uma forma mais eficiente.

In [ ]:
pip install pandas && pip install numpy

Criamos nosso dataframe após pegarmos o conteudo de cada texto

In [ ]:
import pandas as pd
import numpy as np

text_chunks = []

for text in texts:
  text_chunks.append(text.page_content)

df = pd.DataFrame({ 'text_chunks': text_chunks })

É aplicado a função de embedding para cada linha de texto do dataframe, é possível perceber que é um processo demorado, porém, mais para frente iremos resolver esse problema.

In [ ]:
df['ada_embedding'] = df.text_chunks.apply(lambda x: get_embedding(x))
print(df)

                                           text_chunks  \
0                  2023 text-generating language model   
1    Generative Pre-trained Transformer 4 (GPT-4) i...   
2    model created by OpenAI, and the fourth in its...   
3    It was launched on March 14, 2023, and made pu...   
4    product ChatGPT Plus, via OpenAI's API, and vi...   
..                                                 ...   
240  scientist at Hugging Face, argued that the mod...   
241  community due to its closed nature, which prev...   
242  improvements. Hugging Face co-founder Thomas W...   
243  is now a fully closed company with scientific ...   
244                             See also\n\nReferences   

                                         ada_embedding  
0    [-0.03262288123369217, 0.00018029265629593283,...  
1    [-0.00797162763774395, -0.027253426611423492, ...  
2    [0.007487242575734854, -0.012684269808232784, ...  
3    [-0.023756226524710655, -0.025526711717247963,...  
4    [-0.015384607

O mesmo processo será aplicado para a pergunta que o usuário faz

In [ ]:
users_question = "Whats is GPT4 ?"

question_embedding = get_embedding(text=users_question)

Nesta etapa iremos calcular o Cosine e aplicar no nosso dataframe e ordena-lo do maior número para o menor, o resultado irá ser um dataframe onde os textos que estão mais proximos ou mais relacionado com a nossa pergunta no plano multidimensional estã0 no topo da estrura.

In [ ]:
from numpy.linalg import norm


cos_sim = []

for index, row in df.iterrows():
   A = row.ada_embedding
   B = question_embedding

   cosine = np.dot(A,B)/(norm(A)*norm(B))

   cos_sim.append(cosine)

df["cos_sim"] = cos_sim
df.sort_values(by=["cos_sim"], ascending=False)

,text_chunks,ada_embedding,cos_sim
62,GPT-4o\nMain article: GPT-4o,"[0.0025651883333921432, -0.0019583147950470448...",0.880879
202,that GPT-4 was generally an improvement over i...,"[-0.012802904471755028, 0.009636301547288895, ...",0.861737
178,"GPT-4, and has been suggested by Microsoft as ...","[-0.02357708103954792, -0.0160561241209507, -0...",0.851022
52,A 2023 article in Nature stated programmers ha...,"[0.0007326731574721634, -0.0015493525424972177...",0.844072
182,"based on text prompts. With GPT-4, it is able ...","[-0.024939898401498795, -0.00432156166061759, ...",0.838327
...,...,...,...
207,Before being fine-tuned and aligned by reinfor...,"[-0.007887023501098156, -0.008290578611195087,...",0.660931
125,"behavior, such as questions on how to perform ...","[0.006280009169131517, 0.004685619845986366, -...",0.660191
228,"Only a month later, Musk's AI company X.AI acq...","[0.00023362549836747348, -0.014645248651504517...",0.659739
225,AI singularity concerns in an open letter from...,"[0.02828197553753853, -0.023977886885404587, -...",0.659229


## 4. Definir o Modelo utilizado

Para processeguir, é necessário definir um modelo para ser utilizado, o resto da aplicação utilizara LangChain que é um framework utilizado para construir sistemas que utilizam algum LLM, e o modelo utilizado será o text-davinci-003.



### Instalando Depêndencias


In [ ]:
pip install langchain && pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00


### Código

Assim é como o LLM é utilizado de forma simples com o LangChain

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=1)
print(llm("whats is the capital of brazil ?"))






The capital of Brazil is Brasília.


## 5. Definir o formato do Prompt

Agora é necessário definit um prompt, essa parte define o comportamento que as respostas do usuário serão geradas, para o exemplo dado, queremos que nosso código extraia informações da Wikipedia e interaja como uma forma de chat.

Exemplo de formato de prompt: Você é um chatbot e sua função é responder tudo que te perguntam. Responda as questões usando apenas o contexto fornecido, caso esteja incerto de como responder diga Desculpe, mas não sei como te ajudar

Dessa forma, definimos uma limitação que permite o modelo utilizar apenas informações que armazenamos


In [ ]:
context = ""

for index, row in df[0:50].iterrows():
  context = context + " " + row.text_chunks

template = """

  Você é um chatbot e sua função é responder tudo que te perguntam.
  Responda as questões usando apenas o contexto fornecido, caso esteja incerto
  de como responder diga Desculpe, mas não sei como te ajudar

  Contexto:
  {context}

  Question:
  {users_question}

  Answer:

"""

NameError: name 'df' is not defined

Assim, pode-se criar um prompt com as classes do langchain para deixar o processo mais simples.

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])
prompt_text = prompt.format(context = context, users_question=users_question)


ModuleNotFoundError: No module named 'langchain_core'

E por ultimo, chamamos o construtor da classe passando o prompt e verificamos o resultado.

In [ ]:
llm(prompt_text)

## 6. Criando um banco de dados vetor

Um armazenamento de vetor é otimizado para armazenar quantidades grandes de informações que são vetores

Tranformar os textos em vetores é um processo que pode diminuir a velocidade da aplicação, uma alternativa é criar os embeddings e armazernar em um banco de dados de vetores ao invés de criar embeddings toda vez em que um prompt for ser executado.

Vamos testar essa nova alternativo com um novo exemplo:

In [ ]:
import requests
from bs4 import BeautifulSoup


url = 'https://en.wikipedia.org/wiki/Prime_Minister_of_the_United_Kingdom'

response  = requests.get(url)
soup      = BeautifulSoup(response.content, 'html.parser')

text = soup.get_text()
text = text.replace('\n', " ")

Salvamos o resultado em um arquivo

In [ ]:
with open('output.txt', 'w', encoding='utf-8') as file:
  file.write(text)

Certifique-se que os seguintes pacotes estejam instalados.

In [ ]:
pip install langchain && pip install langchain_community && pip install openai && pip install chromadb && pip install tiktoken && pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00


Abrimos agora o arquivo que foi produzido e dividimos junto com a classe RecursiveCharacterTextSplitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('./output.txt', encoding='utf-8') as f:
  text = f.read()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.create_documents([text])
texts[0]

Document(page_content='Prime Minister of the United Kingdom - Wikipedia                                    Jump to content        Main menu      Main menu move to sidebar hide    \t\tNavigation \t   Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate      \t\tContribute \t   HelpLearn to editCommunity portalRecent changesUpload file                    Search            Search                       Appearance                 Create account  Log in         Personal tools       Create account Log')

É definido um model para o embedding e se utiliza os texts chunks junto com o model do embedding para preencher o armazenamento de vetores

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(texts, embeddings)

ModuleNotFoundError: No module named 'langchain'

Agora utiliza a pergunta do usuario e encontra texts chunks que sejam parecidos no nosso armazenamento.

In [ ]:
user_question = "Who is the prime minister of UK ?"

results = db.similarity_search(
    query=user_question
)

results


[Document(page_content='minister of the United Kingdom is the head of government of the United Kingdom. The prime minister advises the sovereign on the exercise of much of the royal prerogative, chairs the Cabinet, and selects its ministers. As modern prime ministers hold office by virtue of their ability to command the confidence of the House of Commons, they sit as members of Parliament. The office of prime minister is not established by any statute or constitutional document, but exists only by long-established'),
 Document(page_content='minister of the United Kingdom is the head of government of the United Kingdom. The prime minister advises the sovereign on the exercise of much of the royal prerogative, chairs the Cabinet, and selects its ministers. As modern prime ministers hold office by virtue of their ability to command the confidence of the House of Commons, they sit as members of Parliament. The office of prime minister is not established by any statute or constitutional doc

Definimo um modelo para o nosso prompt e formatamos ele utilizando a pergunta do usuario e o contexto que conseguimos do armazenamento.

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

template = """

  Você é um chatbot bilingue que gosta de responder perguntas!
  Dado um determinado contexto, responda à questão apenas
  usando o contexto informado. Se você não souber como
  responder, diga: 'Desculpe, mas eu não consigo te
  ajudar com essa pergunta.'

  Contexto:
  {context}

  Question:
  {users_question}

  Resposta:
"""

prompt = PromptTemplate(template=template, input_variable=['context, users_question'])
prompt_text = prompt.format(context=results, users_question=users_question)

Por ultimo é feito a chamada do serviço, e analisado o resultado

In [ ]:
llm(prompt_text)

'  David Cameron'

## Conclusão

O objetivo do artigo foi demonstrar uma abordagem minimalista para o uso de modelos de embeddings, bancos de dados vetoriais e LLMs no processamento de consultas de usuários. Ele mostra como essas tecnologias podem trabalhar em conjunto para fornecer respostas relevantes e precisas, mesmo diante de fatos que mudam constantemente.

Para que o modelo possa responder perguntas sobre nossos próprios dados, utilizamos a abordagem de Injeção de Contexto. Criar um aplicativo LLM com Injeção de Contexto é um processo relativamente simples, mas o principal desafio está na organização e formatação dos dados a serem armazenados em um banco de dados vetorial. Essa etapa é crucial para a recuperação eficiente de informações contextualmente semelhantes e para garantir resultados confiáveis.

Dessa maneira, foi desenvolvida uma aplicação utilizando um Modelo de Linguagem de Grande Escala (LLM) que pode ser ajustado ao contexto e armazenado para otimizar a performance.






Dominik Polzer. (2024). All You Need to Know to Build Your First LLM App
https://readmedium.com/en/https:/towardsdatascience.com/all-you-need-to-know-to-build-your-first-llm-app-eb982c78ffac